In [1]:
import requests
from urllib.parse import unquote
import pandas as pd
from datetime import datetime



def get_options(ticker):

    try :
        geturl=r'https://www.barchart.com/stocks/quotes/AAPL/options'
        apiurl=r'https://www.barchart.com/proxies/core-api/v1/options/get'

        getheaders={

            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'max-age=0',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
            }

        getpay={
            'page': 'all'
        }

        s=requests.Session()
        r=s.get(geturl,params=getpay, headers=getheaders)

        headers={
            'accept': 'application/json',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
            'x-xsrf-token': unquote(unquote(s.cookies.get_dict()['XSRF-TOKEN']))
        }

        payload={
                'fields': 'symbol,baseSymbol,strikePrice,lastPrice,theoretical,volatility,delta,gamma,theta,vega,rho,volume,openInterest,volumeOpenInterestRatio,optionType,daysToExpiration,expirationDate,tradeTime,averageVolatility,historicVolatility30d,baseNextEarningsDate',
                "baseSymbol": str(ticker),
                "groupBy":"optionType",
                "expirationDate":"",
                "meta":"field.shortName,expirations",
                "orderBy":"strikePrice",
                "orderDir":"asc",
                "limit" : 10000
            }
            

        r=s.get(apiurl,params=payload, headers=headers)
        j=r.json()


        # Define DataFrame containing Put and Call option price
        df_call = pd.DataFrame(j['data']['Call'])
        df_put = pd.DataFrame(j['data']['Put'])
        df_merged = pd.concat([df_call, df_put], axis=0)
        df_merged['ExtractTime'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

        return df_merged
    
    except:
        # Create a function to save the ticker that generates an error
        # for the moment we only pass
        pass
    
value = get_options(ticker="GOOGL")
value


In [35]:
URL = "https://symbol-browser.aws.barchart.com/v1/search/instruments"

HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
}
PARAMS = {
    "application":5,
    "query":"A",
    "size":100,
    # "region":"US",
    # "types":"5,7,8,11,15,9,1,3,4,6,23"
}
s=requests.Session()
r=s.get(URL,params=PARAMS, headers=HEADERS)
j=r.json()
data = j['result']['symbols']
instruments = [item['instrument'] for item in data]
df = pd.DataFrame(instruments)
df
# df


KeyError: 'instrument'